In [2]:
from finpie import NewsData #
import pandas as pd
news = NewsData('MSFT', 'microsoft')
df = news.cnbc(datestop = '2018-12-31')
# filter news headlines with a keyword list
# news.filterz = [ 'exxon', 'mobil', 'oil', 'energy' ]
# df = news.filter_data(df)

E:\ProgramData\Anaconda3\lib\site-packages\finpie\news_data\clean_news.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  try:


In [4]:
df.to_csv('cnbc_msft_190101.csv')

In [1]:
# Use bert to compute positive neural negative
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, time, gc
import numpy as np
import pandas as pd

df = pd.read_csv('cnbc_msft_190101.csv')
final = np.array([[0, 0, 0]])
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
base = 350
upper = (len(df) // base) + 1

for i in range(0, upper):
  index1 = base * i
  index2 = base * (i + 1)

  if index2 > len(df):
      df_buf = df.loc[index1:, :]
      headlines_list = list(df_buf.loc[:, 'headline'])
      inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
      outputs = model(**inputs)
      predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
      output = predictions.detach().numpy()

  else:
      df_buf = df.loc[index1:index2, :]
      headlines_list = list(df_buf.loc[:, 'headline'])
      inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
      outputs = model(**inputs)
      predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
      output = predictions.detach().numpy()
      output = output[:-1]


  final = np.concatenate([final, output])
  if index1 == 0:
      final = final[1:]
  del inputs, predictions, tokenizer, model
  gc.collect()
#     time.sleep(5)

  tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
  model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
  print(index1, index2, final.shape)

list_date = list(df['date'])
list_headline = list(df['headline'])

#Formatting the results as a pandas data frame

import pandas as pd
final_n = final.copy()
#Headline #Positive #Negative #Neutral
positive = final_n[:, 0].tolist()
negative = final_n[:, 1].tolist()
neutral = final_n[:, 2].tolist()

table = {'date':list_date,
        'Headline':list_headline,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ['date', "Headline", "Positive", "Negative", "Neutral"])
df.to_csv('BERT_cnbc_msft_190101.csv')


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

0 350 (350, 3)
350 700 (700, 3)
700 1050 (1050, 3)
1050 1400 (1400, 3)
1400 1750 (1750, 3)
1750 2100 (2100, 3)
2100 2450 (2450, 3)
2450 2800 (2800, 3)
2800 3150 (3150, 3)
3150 3500 (3500, 3)
3500 3850 (3850, 3)
3850 4200 (4200, 3)
4200 4550 (4550, 3)
4550 4900 (4900, 3)
4900 5250 (5250, 3)
5250 5600 (5300, 3)
